In [8]:
import math
import pickle
import pandas as pd
from pprint import pprint
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

In [9]:
file_name_uk = '../data/excel/uk-dentsu-music-ryan-matrix-102822.xlsx'
df = pd.DataFrame(pd.read_excel(file_name_uk, skiprows = 1))
df.head()

,Unnamed: 0,Totals,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Sex (D2) [Female],Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Type Of Music/Formats You Regularly Listen To (L6) [Chart Shows],Unnamed: 142,Unnamed: 143,Unnamed: 144,Unnamed: 145,Type Of Music/Formats You Regularly Listen To (L6) [Other],Unnamed: 147,Unnamed: 148,Unnamed: 149,Unnamed: 150
0,NaN,Unwgt,(000s),Vert%,Horz%,Index,Unwgt,(000s),Vert%,Horz%,...,Unwgt,(000s),Vert%,Horz%,Index,Unwgt,(000s),Vert%,Horz%,Index
1,Totals,10811,48441,100,100,100,5378,24294,100,50.15,...,1308,5973,100,12.33,100,814,3685,100,7.61,100
2,Actual Age (D1) [15 or 16 or 17 or 18 or 19 or...,1644,7640,15.77,100,100,788,3654,15.04,47.83,...,284,1300,21.76,17.01,138,138,686,18.61,8.98,118
3,Actual Age (D1) [25 or 26 or 27 or 28 or 29 or...,1844,8763,18.09,100,100,939,4335,17.84,49.47,...,320,1508,25.25,17.21,140,113,554,15.04,6.33,83
4,Actual Age (D1) [36 or 37 or 38 or 39 or 40 or...,1605,7245,14.96,100,100,779,3629,14.94,50.1,...,233,1048,17.55,14.47,117,71,326,8.83,4.49,59


In [10]:
def data_preprocess(file_name):
    df = pd.read_excel(file_name, skiprows = 1, header = [0, 1])
    df.columns = df.columns.map('_'.join)
    df = df.rename(columns = {'Unnamed: 0_level_0_Unnamed: 0_level_1': 'category'})
    df = df.set_index('category')
    df = df.rename(columns = lambda c: c.replace('Sex (D2) [Female]', 'female'))
    df = df.rename(columns = lambda c: c.replace('Sex (D2) [Male]', 'male'))
    return df

def cols_to_keep(df, starts_with, ends_with = ['Horz%', '(000s)']):
    cols_to_keep = []
    for col in list(df.columns):
        for start_with in starts_with:
            for end_with in ends_with:
                if (col.startswith(start_with) and col.endswith(end_with)):
                    cols_to_keep.append(col)
    
    return cols_to_keep

def rows_to_keep(df, starts_with):
    rows_to_keep = []
    for row in list(df.index.values):
        if not isinstance(row, str): continue
        for start_with in starts_with:
            if (row.startswith(start_with)):
                rows_to_keep.append(row)
    return rows_to_keep

def sub_df(df, cols_to_keep, rows_to_keep):
    df = df[cols_to_keep]
    df = df[df.index.isin(rows_to_keep)]
    return df

In [11]:
df = data_preprocess(file_name_uk)
df.head()

,Totals_Unwgt,Totals_(000s),Totals_Vert%,Totals_Horz%,Totals_Index,female_Unwgt,female_(000s),female_Vert%,female_Horz%,female_Index,...,Type Of Music/Formats You Regularly Listen To (L6) [Chart Shows]_Unwgt,Type Of Music/Formats You Regularly Listen To (L6) [Chart Shows]_(000s),Type Of Music/Formats You Regularly Listen To (L6) [Chart Shows]_Vert%,Type Of Music/Formats You Regularly Listen To (L6) [Chart Shows]_Horz%,Type Of Music/Formats You Regularly Listen To (L6) [Chart Shows]_Index,Type Of Music/Formats You Regularly Listen To (L6) [Other]_Unwgt,Type Of Music/Formats You Regularly Listen To (L6) [Other]_(000s),Type Of Music/Formats You Regularly Listen To (L6) [Other]_Vert%,Type Of Music/Formats You Regularly Listen To (L6) [Other]_Horz%,Type Of Music/Formats You Regularly Listen To (L6) [Other]_Index
category,,,,,,,,,,,,,,,,,,,,,
Totals,10811.0,48441.0,100.00,100.0,100.0,5378.0,24294.0,100.00,50.15,100.0,...,1308.0,5973.0,100.00,12.33,100.0,814.0,3685.0,100.00,7.61,100.0
Actual Age (D1) [15 or 16 or 17 or 18 or 19 or 20 or 21 or 22 or 23 or 24],1644.0,7640.0,15.77,100.0,100.0,788.0,3654.0,15.04,47.83,95.0,...,284.0,1300.0,21.76,17.01,138.0,138.0,686.0,18.61,8.98,118.0
Actual Age (D1) [25 or 26 or 27 or 28 or 29 or 30 or 31 or 32 or 33 or 34],1844.0,8763.0,18.09,100.0,100.0,939.0,4335.0,17.84,49.47,99.0,...,320.0,1508.0,25.25,17.21,140.0,113.0,554.0,15.04,6.33,83.0
Actual Age (D1) [36 or 37 or 38 or 39 or 40 or 41 or 42 or 43 or 44],1605.0,7245.0,14.96,100.0,100.0,779.0,3629.0,14.94,50.10,100.0,...,233.0,1048.0,17.55,14.47,117.0,71.0,326.0,8.83,4.49,59.0
Actual Age (D1) [45 or 46 or 47 or 48 or 49 or 50 or 51 or 52 or 53 or 54],1962.0,8740.0,18.04,100.0,100.0,966.0,4426.0,18.22,50.65,101.0,...,201.0,944.0,15.80,10.80,88.0,147.0,658.0,17.85,7.53,99.0


In [12]:
cols_to_keep_gender = cols_to_keep(df, ['Totals_(000s)', 'female', 'male'])
cols_to_keep_age = cols_to_keep(df, ['Totals_(000s)', 'Actual Age (D1)'])

rows_to_keep = rows_to_keep(df, ['Type Of Music/Formats You Regularly Listen To'])

print('Columns for gender table')
pprint(cols_to_keep_gender)
print()
print('Columns for age table')
pprint(cols_to_keep_age)
print()
print('Rows for tables')
pprint(rows_to_keep)

Columns for gender table
['Totals_(000s)', 'female_(000s)', 'female_Horz%', 'male_(000s)', 'male_Horz%']

Columns for age table
['Totals_(000s)',
 'Actual Age (D1) [15 or 16 or 17 or 18 or 19 or 20 or 21 or 22 or 23 or '
 '24]_(000s)',
 'Actual Age (D1) [15 or 16 or 17 or 18 or 19 or 20 or 21 or 22 or 23 or '
 '24]_Horz%',
 'Actual Age (D1) [25 or 26 or 27 or 28 or 29 or 30 or 31 or 32 or 33 or '
 '34]_(000s)',
 'Actual Age (D1) [25 or 26 or 27 or 28 or 29 or 30 or 31 or 32 or 33 or '
 '34]_Horz%',
 'Actual Age (D1) [35 or 36 or 37 or 38 or 39 or 40 or 41 or 42 or 43 or '
 '44]_(000s)',
 'Actual Age (D1) [35 or 36 or 37 or 38 or 39 or 40 or 41 or 42 or 43 or '
 '44]_Horz%',
 'Actual Age (D1) [45 or 46 or 47 or 48 or 49 or 50 or 51 or 52 or 53 or '
 '54]_(000s)',
 'Actual Age (D1) [45 or 46 or 47 or 48 or 49 or 50 or 51 or 52 or 53 or '
 '54]_Horz%',
 'Actual Age (D1) [55 or 56 or 57 or 58 or 59 or 60 or 61 or 62 or 63 or '
 '64]_(000s)',
 'Actual Age (D1) [55 or 56 or 57 or 58 or 59 or

In [13]:
df_gender_cleaned = sub_df(df, cols_to_keep_gender, rows_to_keep)
df_age_cleaned = sub_df(df, cols_to_keep_age, rows_to_keep)

In [14]:
def get_padding(row, data_type):
    i, row_sum = 0, 0
    keys = set()
    for key in dict(row).keys():
        key = key.split('_')[0]
        keys.add(key) 
    keys.remove('Totals')
    for key in keys:
        row_sum += row[key + '_Horz%']
        i += 1
    return round((100.0 - row_sum) / i, 2)

# with open( '../data/mappings/music_all_mapping.pkl', 'rb') as f:
#     music_all_mapping = pickle.load(f)
# pprint(music_all_mapping) 
# print()
# with open( '../data/mappings/other_all_mapping.pkl', 'rb') as f:
#     other_all_mapping = pickle.load(f)
# pprint(other_all_mapping)    

In [15]:
def create_df_gender(df, country_code = 'UK', content_format_source_id = 3, geo_item_id = None):
    now = datetime.now()
    table = {
        'ContentFormatId': [],
        'CategoryDB': [],
        'CategoryData': [],
        'CountryCode': [],
        'IsMale': [],
        'Percentage': [],
        'MaleCount': [],
        'FemaleCount': [],
        'TotalCount': []
    }
    
    with open( '../data/mappings/music_music_mapping_UK.pkl', 'rb') as f:
        mappings = pickle.load(f)
    # with open('../data/mappings/other_all_mapping.pkl', 'rb') as f:
    #     other_all_mapping = pickle.load(f)
    # mappings = {**music_all_mapping, **other_all_mapping}
    
    for index, row in df.iterrows():
        mapping = mappings[index]
        
        table['ContentFormatId'].extend([mapping['id']] * 2)
        table['CategoryDB'].extend([mapping['table']] * 2)
        table['CategoryData'].extend([index] * 2)
        table['CountryCode'].extend([country_code] * 2)
        
        padding = get_padding(row, 'gender')
        table['IsMale'].append(False)
        table['Percentage'].append(row['female_Horz%'] + padding)
        table['FemaleCount'].append(row['female_(000s)'])
        table['MaleCount'].append(0)
        
        table['IsMale'].append(True)
        table['Percentage'].append(row['male_Horz%'] + padding)
        table['FemaleCount'].append(0)
        table['MaleCount'].append(row['male_(000s)'])
        
        table['TotalCount'].extend([row['male_(000s)'] + row['female_(000s)']] * 2)
    df = pd.DataFrame.from_dict(table)
    return df

In [16]:
df_gender_cleaned.head()

,Totals_(000s),female_(000s),female_Horz%,male_(000s),male_Horz%
category,,,,,
Type Of Music/Formats You Regularly Listen To (L6) [80s Format/Oldies (e.g. Michael Jackson)],13405.0,7093.0,52.91,6243.0,46.57
Type Of Music/Formats You Regularly Listen To (L6) [Alternative/Rock (e.g. Foo Fighters)],8987.0,3838.0,42.71,5053.0,56.23
Type Of Music/Formats You Regularly Listen To (L6) [Christian/Inspirational/Religious],1821.0,827.0,45.39,994.0,54.61
Type Of Music/Formats You Regularly Listen To (L6) [Classic Rock/Classic Hits],11692.0,5101.0,43.63,6514.0,55.71
Type Of Music/Formats You Regularly Listen To (L6) [Classical],6045.0,2668.0,44.13,3348.0,55.38


In [17]:
df_gender_pre = create_df_gender(df_gender_cleaned, country_code = 'UK')
df_gender_pre.head()

,ContentFormatId,CategoryDB,CategoryData,CountryCode,IsMale,Percentage,MaleCount,FemaleCount,TotalCount
0,342,Adult Album Alternative,Type Of Music/Formats You Regularly Listen To ...,UK,False,53.17,0.0,7093.0,13336.0
1,342,Adult Album Alternative,Type Of Music/Formats You Regularly Listen To ...,UK,True,46.83,6243.0,0.0,13336.0
2,365,Alternative Rock,Type Of Music/Formats You Regularly Listen To ...,UK,False,43.24,0.0,3838.0,8891.0
3,365,Alternative Rock,Type Of Music/Formats You Regularly Listen To ...,UK,True,56.76,5053.0,0.0,8891.0
4,361,Religious (Music and Audio),Type Of Music/Formats You Regularly Listen To ...,UK,False,45.39,0.0,827.0,1821.0


In [18]:
def get_row_info(row, min_age, max_age, padding):
    keys = set()
    for key in dict(row).keys():
        key = key.split('_')[0]
        keys.add(key)
    total = row['Totals_(000s)']
    for key in keys:
        if(str(min_age) in key or str(max_age) in key):
            percentage_with_padding = row[key + '_Horz%'] + padding
            return percentage_with_padding, round(total * percentage_with_padding / 100, 0)
    
def get_df_age(df, country_code = 'UK', content_format_source_id = 3, geo_item_id = None):
    now = datetime.now()
    table = {
        'ContentFormatId': [],
        'CategoryDB': [],
        'CategoryData': [],
        'CountryCode': [],
        'MinAge': [],
        'MaxAge': [],
        'Percentage': [],
        'Count': [],
        'TotalCount': []
    }
    
    with open( '../data/mappings/music_music_mapping_UK.pkl', 'rb') as f:
        mappings = pickle.load(f)
    
    for index, row in df.iterrows():
        total_count = 0
        mapping = mappings[index]
        
        table['ContentFormatId'].extend([mapping['id']] * 6)
        table['CategoryDB'].extend([mapping['table']] * 6)
        table['CategoryData'].extend([index] * 6)
        table['CountryCode'].extend([country_code] * 6)
        
        padding = get_padding(row, 'age')
        
        table['MinAge'].append(0)
        table['MaxAge'].append(24)
        percentage, count = get_row_info(row, None, 24, padding)
        total_count += count
        table['Percentage'].append(percentage)
        table['Count'].append(count)
        
        table['MinAge'].append(25)
        table['MaxAge'].append(34)
        percentage, count = get_row_info(row, 25, 34, padding)
        total_count += count
        table['Percentage'].append(percentage)
        table['Count'].append(count)
        
        table['MinAge'].append(35)
        table['MaxAge'].append(44)
        percentage, count = get_row_info(row, 35, 44, padding)
        total_count += count
        table['Percentage'].append(percentage)
        table['Count'].append(count)
        
        table['MinAge'].append(45)
        table['MaxAge'].append(54)
        percentage, count = get_row_info(row, 45, 54, padding)
        total_count += count
        table['Percentage'].append(percentage)
        table['Count'].append(count)
        
        table['MinAge'].append(55)
        table['MaxAge'].append(64)
        percentage, count = get_row_info(row, 55, 64, padding)
        total_count += count
        table['Percentage'].append(percentage)
        table['Count'].append(count)
        
        table['MinAge'].append(65)
        table['MaxAge'].append(100)
        percentage, count = get_row_info(row, 65, None, padding)
        total_count += count
        table['Percentage'].append(percentage)
        table['Count'].append(count)
        
        table['TotalCount'].extend([total_count] * 6)
        
    df = pd.DataFrame.from_dict(table)
    return df

In [19]:
df_age_cleaned.head()

,Totals_(000s),Actual Age (D1) [15 or 16 or 17 or 18 or 19 or 20 or 21 or 22 or 23 or 24]_(000s),Actual Age (D1) [15 or 16 or 17 or 18 or 19 or 20 or 21 or 22 or 23 or 24]_Horz%,Actual Age (D1) [25 or 26 or 27 or 28 or 29 or 30 or 31 or 32 or 33 or 34]_(000s),Actual Age (D1) [25 or 26 or 27 or 28 or 29 or 30 or 31 or 32 or 33 or 34]_Horz%,Actual Age (D1) [35 or 36 or 37 or 38 or 39 or 40 or 41 or 42 or 43 or 44]_(000s),Actual Age (D1) [35 or 36 or 37 or 38 or 39 or 40 or 41 or 42 or 43 or 44]_Horz%,Actual Age (D1) [45 or 46 or 47 or 48 or 49 or 50 or 51 or 52 or 53 or 54]_(000s),Actual Age (D1) [45 or 46 or 47 or 48 or 49 or 50 or 51 or 52 or 53 or 54]_Horz%,Actual Age (D1) [55 or 56 or 57 or 58 or 59 or 60 or 61 or 62 or 63 or 64]_(000s),Actual Age (D1) [55 or 56 or 57 or 58 or 59 or 60 or 61 or 62 or 63 or 64]_Horz%,Actual Age (D1) [65 or 66 or 67 or 68 or 69 or 70 or 71 or 72 or 73 or 74 or 75]_(000s),Actual Age (D1) [65 or 66 or 67 or 68 or 69 or 70 or 71 or 72 or 73 or 74 or 75]_Horz%
category,,,,,,,,,,,,,
Type Of Music/Formats You Regularly Listen To (L6) [80s Format/Oldies (e.g. Michael Jackson)],13405.0,1536,11.46,1756,13.1,2414,18.01,3637,27.13,2578,19.23,1484,11.07
Type Of Music/Formats You Regularly Listen To (L6) [Alternative/Rock (e.g. Foo Fighters)],8987.0,1621,18.04,1896,21.1,1931,21.49,1736,19.31,1296,14.42,506,5.63
Type Of Music/Formats You Regularly Listen To (L6) [Christian/Inspirational/Religious],1821.0,362,19.89,451,24.77,410,22.51,236,12.94,179,9.83,183,10.07
Type Of Music/Formats You Regularly Listen To (L6) [Classic Rock/Classic Hits],11692.0,1329,11.37,1658,14.18,1818,15.55,2395,20.48,2453,20.98,2038,17.43
Type Of Music/Formats You Regularly Listen To (L6) [Classical],6045.0,654,10.82,880,14.55,1012,16.74,949,15.7,1183,19.57,1368,22.62


In [20]:
df_age_pre = get_df_age(df_age_cleaned, country_code = 'UK')
df_age_pre.head(12)

,ContentFormatId,CategoryDB,CategoryData,CountryCode,MinAge,MaxAge,Percentage,Count,TotalCount
0,342,Adult Album Alternative,Type Of Music/Formats You Regularly Listen To ...,UK,0,24,11.46,1536.0,13405.0
1,342,Adult Album Alternative,Type Of Music/Formats You Regularly Listen To ...,UK,25,34,13.10,1756.0,13405.0
2,342,Adult Album Alternative,Type Of Music/Formats You Regularly Listen To ...,UK,35,44,18.01,2414.0,13405.0
3,342,Adult Album Alternative,Type Of Music/Formats You Regularly Listen To ...,UK,45,54,27.13,3637.0,13405.0
4,342,Adult Album Alternative,Type Of Music/Formats You Regularly Listen To ...,UK,55,64,19.23,2578.0,13405.0
5,342,Adult Album Alternative,Type Of Music/Formats You Regularly Listen To ...,UK,65,100,11.07,1484.0,13405.0
6,365,Alternative Rock,Type Of Music/Formats You Regularly Listen To ...,UK,0,24,18.04,1621.0,8985.0
7,365,Alternative Rock,Type Of Music/Formats You Regularly Listen To ...,UK,25,34,21.10,1896.0,8985.0
8,365,Alternative Rock,Type Of Music/Formats You Regularly Listen To ...,UK,35,44,21.49,1931.0,8985.0
9,365,Alternative Rock,Type Of Music/Formats You Regularly Listen To ...,UK,45,54,19.31,1735.0,8985.0


In [21]:
df_gender_pre.to_csv('../data/df_gender_pre_UK.csv')
df_age_pre.to_csv('../data/df_age_pre_UK.csv')

In [22]:
def unite_common_categories_gender(df, len_group = 2):
    common_content_id = []
    common_content = {
        'ContentFormatId': [],
        'CategoryDB': [],
        'CategoryData': [],
        'CountryCode': [],
        'IsMale': [],
        'Percentage': [],
        'MaleCount': [],
        'FemaleCount': [],
        'TotalCount': []
    }
    content_groups = df.groupby('ContentFormatId')
    for content_id, content_group in content_groups:
        if (len(content_group) > len_group):
            common_content_id.append(content_id)
            
            common_content['ContentFormatId'].extend([content_id] * 2)
            common_content['CategoryDB'].extend([content_group['CategoryDB'].values[0]] * 2)
            common_content['CategoryData'].extend([content_group['CategoryData'].values[0]] * 2)
            common_content['CountryCode'].extend([content_group['CountryCode'].values[0]] * 2)
        
            content_sub_groups = content_group.groupby('IsMale')
            female_count = content_sub_groups['FemaleCount'].sum().values[0]
            male_count = content_sub_groups['MaleCount'].sum().values[1]
            total = content_sub_groups['TotalCount'].sum().values[1]
            
            common_content['IsMale'].append(False)
            common_content['Percentage'].append(round(female_count / total * 100, 2))
            common_content['FemaleCount'].append(female_count)
            common_content['MaleCount'].append(0)
            
            common_content['IsMale'].append(True)
            common_content['Percentage'].append(round(male_count / total * 100, 2))
            common_content['FemaleCount'].append(0)
            common_content['MaleCount'].append(male_count)
            
            common_content['TotalCount'].extend([total] * 2)
    
    df = df[~df['ContentFormatId'].isin(common_content_id)]
    df_new = pd.DataFrame.from_dict(common_content)
    df = pd.concat([df, df_new], ignore_index = True)
    df.to_csv('../data/df_gender_NL.csv')
    return df

def unite_common_categories_age(df, len_group = 6):
    common_content_id = []
    common_content = {
        'ContentFormatId': [],
        'CategoryDB': [],
        'CategoryData': [],
        'CountryCode': [],
        'MinAge': [],
        'MaxAge': [],
        'Percentage': [],
        'Count': [],
        'TotalCount': []
    }
    
    content_groups = df.groupby('ContentFormatId')
    for content_id, content_group in content_groups:
        if (len(content_group) > len_group):
            common_content_id.append(content_id)
            
            common_content['ContentFormatId'].extend([content_id] * 6)
            common_content['CategoryDB'].extend([content_group['CategoryDB'].values[0]] * 6)
            common_content['CategoryData'].extend([content_group['CategoryData'].values[0]] * 6)
            common_content['CountryCode'].extend([content_group['CountryCode'].values[0]] * 6)
            
            content_sub_groups = content_group.groupby('MinAge', dropna = False)
            
            count_and_total = content_sub_groups[['Count', 'TotalCount']].sum().values

            common_content['MinAge'].append(0)
            common_content['MaxAge'].append(24)
            common_content['Percentage'].append(round(count_and_total[0][0] / count_and_total[0][1], 2))
            common_content['Count'].append(count_and_total[0][0])
            
            common_content['MinAge'].append(25)
            common_content['MaxAge'].append(34)
            common_content['Percentage'].append(round(count_and_total[1][0] / count_and_total[1][1], 2))
            common_content['Count'].append(count_and_total[1][0])
            
            common_content['MinAge'].append(35)
            common_content['MaxAge'].append(44)
            common_content['Percentage'].append(round(count_and_total[2][0] / count_and_total[2][1], 2))
            common_content['Count'].append(count_and_total[2][0])
            
            common_content['MinAge'].append(45)
            common_content['MaxAge'].append(54)
            common_content['Percentage'].append(round(count_and_total[3][0] / count_and_total[3][1], 2))
            common_content['Count'].append(count_and_total[3][0])
            
            common_content['MinAge'].append(55)
            common_content['MaxAge'].append(64)
            common_content['Percentage'].append(round(count_and_total[4][0] / count_and_total[4][1], 2))
            common_content['Count'].append(count_and_total[4][0])
            
            common_content['MinAge'].append(65)
            common_content['MaxAge'].append(100)
            common_content['Percentage'].append(round(count_and_total[5][0] / count_and_total[5][1], 2))
            common_content['Count'].append(count_and_total[5][0])
            
            common_content['TotalCount'].extend([count_and_total[0][1]] * 6)
            
    df = df[~df['ContentFormatId'].isin(common_content_id)]
    df_new = pd.DataFrame.from_dict(common_content)
    df = pd.concat([df, df_new], ignore_index = True)
    df.to_csv('../data/df_age_NL.csv')
    return df


In [23]:
df_age = unite_common_categories_age(df_age_pre)
df_age = df_age.drop(['CategoryDB', 'CategoryData', 'Count', 'TotalCount'], axis = 1)
df_age.to_csv('../data/table/df_age_UK_table.csv')
df_age

,ContentFormatId,CountryCode,MinAge,MaxAge,Percentage
0,342,UK,0,24,11.46
1,342,UK,25,34,13.10
2,342,UK,35,44,18.01
3,342,UK,45,54,27.13
4,342,UK,55,64,19.23
...,...,...,...,...,...
97,375,UK,25,34,0.12
98,375,UK,35,44,0.17
99,375,UK,45,54,0.21
100,375,UK,55,64,0.21


In [24]:
df_gender = unite_common_categories_gender(df_gender_pre)
df_gender = df_gender.drop(['CategoryDB', 'CategoryData', 'MaleCount', 'FemaleCount', 'TotalCount'], axis = 1)
df_gender.to_csv('../data/table/df_gender_UK_table.csv')
df_gender

,ContentFormatId,CountryCode,IsMale,Percentage
0,342,UK,False,53.17
1,342,UK,True,46.83
2,365,UK,False,43.24
3,365,UK,True,56.76
4,361,UK,False,45.39
5,361,UK,True,54.61
6,345,UK,False,43.96
7,345,UK,True,56.04
8,346,UK,False,44.37
9,346,UK,True,55.62
